<a href="https://colab.research.google.com/github/hiu04/Data-Science/blob/main/MMAI5200_midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Question 1

In [ ]:
import numpy as np

In [ ]:
def simplex(c1, c2, a11, a12, a21, a22, b1, b2):
    # Construct the objective function coefficients
    c = np.array([-c1, -c2])  # Negative values to obtain maximization

    # Construct the constraint matrix A
    A = np.array([[a11, a12],
                  [a21, a22]])

    # Construct the right-hand side of the constraints
    b = np.array([b1, b2])

    # Initialize the sizes
    basicSize = A.shape[0] # Size of variables currently using
    nonbasicSize = A.shape[1] - basicSize # Size of variables currently not using

    # Initialize the index tracker for coefficients (array c)
    cindx = [i for i in range(len(c))]

    # Initialize basic and nonbasic variable coefficients
    cbT = np.array(c[nonbasicSize:]) # basic
    cnT = np.array(c[:nonbasicSize]) # nonbasic

    # Iteration of Simplex Algorithm till optimization
    while True:
        cbIndx = cindx[nonbasicSize:]
        cnIndx = cindx[:nonbasicSize]

        B = A[:, cbIndx]
        Binv = np.linalg.inv(B)
        N = A[:, cnIndx]

        bHat = Binv @ b
        yT = cbT @ Binv
        cnHat = cnT - (yT @ N)

        if all(i >= 0 for i in cnHat):
            # Extract the solution for x1 and x2
            x1, x2 = 0, 0
            for i, index in enumerate(cbIndx):
                if index == 0:
                    x1 = bHat[i]
                elif index == 1:
                    x2 = bHat[i]
            z = c1 * x1 + c2 * x2
            return x1, x2, z

        cnMinIndx = np.argmin(cnHat)
        indx = cindx[cnMinIndx]
        Ahat = Binv @ A[:, indx]

        ratios = []
        for i in range(len(bHat)):
            Aval = Ahat[i]
            Bval = bHat[i]
            if Aval <= 0:
                ratios.append(float('inf'))
            else:
                ratios.append(Bval / Aval)

        ratioMinIndx = np.argmin(ratios)
        cnT[cnMinIndx], cbT[ratioMinIndx] = cbT[ratioMinIndx], cnT[cnMinIndx]
        cindx[cnMinIndx], cindx[ratioMinIndx + nonbasicSize] = \
        cindx[ratioMinIndx + nonbasicSize], cindx[cnMinIndx]


In [ ]:
# Test the function with the class example
x1, x2, z = simplex(100, 150, 3, 2, 1, 2, 6, 5)
print("Optimal solution: \nz =", round(z,1), "\nx1 =", round(x1,1),"\nx2 =", x2)

Optimal solution: 
z = 387.5 
x1 = 0.5 
x2 = 2.25


## Question 2

In [ ]:
def branch_and_bound(c1, c2, a11, a12, a21, a22, b1, b2):
    # Integrate Simplex Function from Part 1
    x1, x2, z = simplex(c1, c2, a11, a12, a21, a22, b1, b2)

    # Adjustments to ensure integer solutions
    x1_int = int(x1)
    x2_int = int(x2)

    # Recalculate the optimal z value with the integers
    z_int = c1 * x1_int + c2 * x2_int
    print(f"Optimal Solution: \nz = {z_int}\nx1 = {x1_int}\nx2 = {x2_int}")

In [ ]:
# Test the function with the class example
branch_and_bound(100, 150, 3, 2, 1, 2, 6, 5)

Optimal Solution: 
z = 300
x1 = 0
x2 = 2


Notice that the only difference between Simplex Algorithm and the Branch-and-Bound Algorithm is the constraints for $x_{1}$ and $x_{2}$. In this question, where using Branch-and-Bound Algorithm, $x_{1}$ and $x_{2}$ are positive integer while Simplex Algorithm only needs positive values. As $x_{1}$ and $x_{2}$ are now some optimal positive integers, the result of optimal $z$ value has changed to 300 now with $x_{1} = 0$ and $x_{2} = 2$ using expression $z = 100x_{1} + 150x_{2}$.

## Question 3

Let $x_{ij}$ be the quantity of product transported from $\text{Warehouse}_{i}$ to $\text{City}_{j}$.

- Capacity constraints for warehouses:

The capacity of Warehouse 1 is 500 units, and the capacity of Warehourse 2 is 1800 units.

$$x_{11} + x_{12} + x_{13} \leq 500 \quad \text{(Warehouse 1)}$$
$$x_{21} + x_{22} + x_{23} \leq 1800 \quad \text{(Warehouse 2)}$$

- Demand constraints for cities:

The demand values for the product are 600, 700, and 800 units in City 1, 2, and 3, respectively.

$$x_{11} + x_{21} = 600 \quad \text{(City 1)}$$
$$x_{12} + x_{22} = 700 \quad \text{(City 2)}$$
$$x_{13} + x_{23} = 300 \quad \text{(City 3)}$$

- Non-negative quantity of product:

$$x_{ij} \geq 0 \ \text{for i = 1,2 and j = 1,2,3}$$


**Goal**

Minimize the total transportation cost $Z$:
$$Z = 2x_{11} + 1.5x_{12} + 10x_{13}+4x_{21} + 3.5x_{22} + 6x_{23}$$

In [ ]:
! pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp

In [ ]:
def transportation_cost(capacities, demands, costs):

    # Create the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Define variables x_ij
    x = {}
    for i in range(len(capacities)):
        for j in range(len(demands)):
            # Get non-negative integers for x_ij
            x[(i, j)] = solver.IntVar(0, solver.infinity(), f'x[{i},{j}]')

    # Define constraints
    # Capacity constraints
    for i in range(len(capacities)):
        solver.Add(sum(x[(i, j)] for j in range(len(demands))) <= capacities[i])

    # Demand constraints
    for j in range(len(demands)):
        solver.Add(sum(x[(i, j)] for i in range(len(capacities))) == demands[j])

    # Define objective, the minimization of cost
    objective = solver.Objective()
    for i in range(len(capacities)):
        for j in range(len(demands)):
            objective.SetCoefficient(x[(i, j)], costs[i][j])
    objective.SetMinimization()

    # Solve the problem
    status = solver.Solve()
    from tabulate import tabulate

    if status == pywraplp.Solver.OPTIMAL:
        total_cost = 0
        table_data = []
        for i in range(len(capacities)):
            for j in range(len(demands)):
                units = x[(i, j)].solution_value()
                table_data.append([f'Warehouse {i+1}', f'City {j+1}',
                                   f'{units} units'])
                total_cost += costs[i][j] * units
        print(tabulate(table_data, headers=['From', 'To', 'Units'],
                       tablefmt='grid'))
        print(f'\nTotal cost: ${total_cost}')
    else:
        print('The problem does not have an optimal solution.')

In [ ]:
capacities = [500, 1800]
demands = [600, 700, 300]
costs = [[2, 1.5, 10],[4, 3.5, 6]]

In [ ]:
transportation_cost(capacities, demands, costs)

+-------------+--------+-------------+
| From        | To     | Units       |
+=============+========+=============+
| Warehouse 1 | City 1 | 0.0 units   |
+-------------+--------+-------------+
| Warehouse 1 | City 2 | 500.0 units |
+-------------+--------+-------------+
| Warehouse 1 | City 3 | 0.0 units   |
+-------------+--------+-------------+
| Warehouse 2 | City 1 | 600.0 units |
+-------------+--------+-------------+
| Warehouse 2 | City 2 | 200.0 units |
+-------------+--------+-------------+
| Warehouse 2 | City 3 | 300.0 units |
+-------------+--------+-------------+

Total cost: $5650.0
